<a href="https://colab.research.google.com/github/GRACOPORDEUS/kaggle/blob/main/%5B202205%5D%20Kaggle_State_of_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import pandas as pd
import numpy as np
from enum import Enum
from typing import List, Type, Tuple
from abc import ABC, abstractmethod


import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.express as px
from plotly.subplots import make_subplots
from collections import namedtuple 


from google.colab import drive
import os

#### Importando dataset

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
os.chdir("/content/gdrive/MyDrive/kaggle")

In [ ]:
os.environ['KAGGLE_USERNAME'] = "glaucopordeus" # username from the json file
os.environ['KAGGLE_KEY'] = "180e7c7de75760d5ad7809224f8bb918" # key from the json file

In [ ]:
! kaggle datasets download -d datahackers/state-of-data-2021

  0% 0.00/539k [00:00<?, ?B/s]
100% 539k/539k [00:00<00:00, 50.7MB/s]


In [ ]:
! unzip state-of-data-2021.zip

Archive:  state-of-data-2021.zip
  inflating: State of Data 2021 - Dataset - Pgina1.csv  


In [ ]:
# https://www.kaggle.com/code/andresionek/what-makes-a-kaggler-valuable/notebook
# https://www.kaggle.com/datasets/datahackers/state-of-data-2021
# https://www.roberthalf.com.br/blog/salario/guia-salarial-2021-da-robert-half-ja-esta-disponivel
# https://www.kaggle.com/andresionek/code
# https://www.kaggle.com/datasets/kaggle/kaggle-survey-2018/discussion/74297
# https://www.kaggle.com/code/artvolgin/exploring-phd-community-with-network-analysis/notebook


In [ ]:
df = pd.read_csv('State of Data 2021 - Dataset - Pgina1.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645 entries, 0 to 2644
Columns: 356 entries, ('P0', 'id') to ('P9_a_g ', 'Ainda não conhecia o Data Hackers')
dtypes: float64(293), int64(7), object(56)
memory usage: 7.2+ MB


#### Funções

Base

In [ ]:
class Mapping(Enum):
    """
    Contains dicts mapping values found in the surveys to values we want to replace with.
    """
    COMPENSATION={ 
        '$0-999': '0-10k',
        '1,000-1,999': '0-10k',
        '2,000-2,999': '0-10k',
        '3,000-3,999': '0-10k',
        '4,000-4,999': '0-10k',
        '5,000-7,499': '0-10k',
        '7,500-9,999': '0-10k',
        '10,000-14,999': '10-20k',
        '15,000-19,999': '10-20k',
        '20,000-24,999': '20-30k',
        '25,000-29,999': '20-30k',
        '30,000-39,999': '30-40k',
        '40,000-49,999': '40-50k',
        '50,000-59,999': '50-60k',
        '60,000-69,999': '60-70k',
        '70,000-79,999': '70-80k',
        '80,000-89,999': '80-90k',
        '90,000-99,999': '90-100k',
        '100,000-124,999': '100-125k',
        '125,000-149,999': '125-150k',
        '150,000-199,999': '150-200k',
        '200,000-249,999': '200-250k',
        '300,000-500,000': '300-500k',
        '> $500,000': np.nan,
        '0-10,000': '0-10k',
        '10-20,000': '10-20k',
        '20-30,000': '20-30k',
        '30-40,000': '30-40k',
        '40-50,000': '40-50k',
        '50-60,000': '50-60k',
        '60-70,000': '60-70k',
        '70-80,000': '70-80k',
        '80-90,000': '80-90k',
        '90-100,000': '90-100k',
        '100-125,000': '100-125k',
        '125-150,000': '125-150k',
        '150-200,00': '150-200k',
        '200-250,000': '200-250k',
        '300-400,000': '300-500k',
        '400-500,000': '300-500k',
        '500,000+': np.nan,
        'I do not wish to disclose my approximate yearly compensation': np.nan
    }
    JOB_TITLE={
        'Data Scientist': 'Data Scientist',
        'Software Engineer': 'Software Engineer',
        'Data Analyst': 'Data Analyst',
        'Other': 'Other',
        'Research Scientist': 'Research Scientist/Statistician',
        'Business Analyst': 'Business Analyst',
        'Product/Project Manager': 'Product/Project Manager',
        'Data Engineer': 'Data Engineer/DBA',
        'Not employed': 'Currently not employed',
        'Machine Learning Engineer': 'Machine Learning Engineer',
        'Statistician': 'Research Scientist/Statistician',
        'Consultant': 'Other',
        'Research Assistant': 'Research Scientist/Statistician',
        'Manager': 'Manager/C-level',
        'DBA/Database Engineer': 'Data Engineer/DBA',
        'Chief Officer': 'Manager/C-level',
        'Developer Advocate': 'Other',
        'Marketing Analyst': 'Business Analyst',
        'Salesperson': 'Other',
        'Principal Investigator': 'Research Scientist/Statistician',
        'Data Journalist': 'Other',
        'Currently not employed': 'Currently not employed', 
        'Student': 'Student'
    } 
    GENDER={
        'Male': 'Men',
        'Female': 'Women',
        'Man': 'Men',
        'Woman': 'Women',
        'Prefer not to say': np.nan, # Very few answers on those categories to do any meaningful analysis
        'Prefer to self-describe':  np.nan, # Very few answers on those categories to do any meaningful analysis
        'Nonbinary':  np.nan # Very few answers on those categories to do any meaningful analysis
    }
    AGE={
        '18-21': '18-21', 
        '22-24': '22-24', 
        '25-29': '25-29',
        '30-34': '30-34', 
        '35-39': '35-39', 
        '40-44': '40-44', 
        '45-49': '45-49', 
        '50-54': '50-54', 
        '55-59': '55-59', 
        '60-69': '60-69', 
        '70+': '70+',
        '70-79': '70+',
        '80+': '70+'
    }
    EDUCATION={
        'Master’s degree': 'Master’s', 
        'Bachelor’s degree': 'Bachelor’s',
        'Some college/university study without earning a bachelor’s degree': 'Some college',
        'Doctoral degree': 'Doctoral',
        'Professional degree': 'Professional',
        'I prefer not to answer': np.nan,
        'No formal education past high school': 'High school'
    }
    YEARS_WRITING_CODE={
        '3-5 years': '3-5 years',
        '1-2 years': '1-3 years',
        '2-3 years': '1-3 years',
        '5-10 years': '5-10 years',
        '10-20 years': '10+ years',
        '< 1 years': '< 1 year',
        '< 1 year': '< 1 year',
        '20+ years': '10+ years',
        np.nan: 'None',
        'I have never written code': 'None',
        'I have never written code but I want to learn': 'None',
        '20-30 years': '10+ years',
        '30-40 years': '10+ years',
        '40+ years': '10+ years'
    }    
    YEARS_WRITING_CODE_PROFILES={
        '3-5 years': '3-10 years',
        '1-2 years': '1-2 years',
        '2-3 years': '2-3 years',
        '5-10 years': '3-10 years',
        '10-20 years': '10+ years',
        '< 1 years': '0-1 years',
        '< 1 year': '0-1 years',
        '20+ years': '10+ years',
        np.nan: 'None',
        'I have never written code': 'None',
        'I have never written code but I want to learn': 'None',
        '20-30 years': '10+ years',
        '30-40 years': '10+ years',
        '40+ years': '10+ years'
    } 
    RECOMMENDED_LANGUAGE={
        'Python': 'Python',
        'R': 'R',
        'SQL': 'SQL',
        'C++': 'C++',
        'MATLAB': 'MATLAB',
        'Other': 'Other',
        'Java': 'Java',
        'C': 'C',
        'None': 'None',
        'Javascript': 'Javascript',
        'Julia': 'Julia',
        'Scala': 'Other',
        'SAS': 'Other',
        'Bash': 'Bash',
        'VBA': 'Other',
        'Go': 'Other',
        'Swift': 'Swift',
        'TypeScript': 'Other'
    } 
    LANGUAGES={
        'SQL': 'SQL', 
        'R': 'R', 
        'Java': 'Java', 
        'MATLAB': 'MATLAB', 
        'Python': 'Python', 
        'Javascript/Typescript': 'Javascript/Typescript',
        'Bash': 'Bash', 
        'Visual Basic/VBA': 'VBA', 
        'Scala': 'Scala', 
        'PHP': 'Other', 
        'C/C++': 'C/C++',
        'Other': 'Other', 
        'C#/.NET': 'Other',
        'Go': 'Other', 
        'SAS/STATA': 'Other', 
        'Ruby': 'Other', 
        'Julia': 'Julia',
        'None': 'None',
         np.nan: 'None',
        'Javascript': 'Javascript/Typescript',
        'C': 'C/C++', 
        'TypeScript': 'Javascript/Typescript', 
        'C++': 'C/C++', 
        'Swift': 'Swift'
    }
    YEARS_USING_ML={
        '1-2 years': '1-3 years',
        '2-3 years': '1-3 years',
        '< 1 year': '< 1 year',
        'Under 1 year': '< 1 year',
        '< 1 years': '< 1 year',
        '3-4 years': '3-5 years',
        '5-10 years': '5+ years',
        '4-5 years': '3-5 years',
        np.nan: 'None',
        'I have never studied machine learning but plan to learn in the future': 'None',
        'I do not use machine learning methods': 'None',
        '10-15 years': '5+ years',
        '20+ years': '5+ years',
        '10-20 years': '5+ years',
        '20 or more years': '5+ years',
        'I have never studied machine learning and I do not plan to': 'None'
    } 
    YEARS_USING_ML_PROFILES={
        '1-2 years': '1-2 years',
        '2-3 years': '2-3 years',
        '< 1 year': '0-1 years',
        'Under 1 year': '0-3 years',
        '< 1 years': '0-1 years',
        '3-4 years': '3-10 years',
        '5-10 years': '3-10 years',
        '4-5 years': '3-10 years',
        np.nan: 'None',
        'I have never studied machine learning but plan to learn in the future': 'None',
        'I do not use machine learning methods': 'None',
        '10-15 years': '10+ years',
        '20+ years': '10+ years',
        '10-20 years': '10+ years',
        '20 or more years': '10+ years',
        'I have never studied machine learning and I do not plan to': 'None'
    } 
    PRIMARY_TOOL={
        'Local development environments (RStudio, JupyterLab, etc.)': 'Local or hosted development environments',
        'Basic statistical software (Microsoft Excel, Google Sheets, etc.)': 'Basic statistical software',
        'Local or hosted development environments (RStudio, JupyterLab, etc.)': 'Local or hosted development environments',
        'Cloud-based data software & APIs (AWS, GCP, Azure, etc.)': 'Cloud-based data software & APIs',
        'Other': 'Other',
        'Advanced statistical software (SPSS, SAS, etc.)': 'Advanced statistical software',
        'Business intelligence software (Salesforce, Tableau, Spotfire, etc.)': 'Business intelligence software'
    }
    COUNTRY = {
        'India': 'India',
        'United States of America': 'United States',
        'Other': 'Other',
        'Brazil': 'Brazil',
        'Russia': 'Russia',
        'Japan': 'Japan',  
        'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
        'Germany': 'Germany',
        'China': 'China',
        'Spain': 'Spain',
        'France': 'France',
        'Canada': 'Canada',
        'Italy': 'Italy',
        'Nigeria': 'Nigeria',
        'Turkey': 'Turkey',
        'Australia': 'Australia'
    }
    IDE={
        'None': 'None', 
        'MATLAB': 'MATLAB', 
        'RStudio': 'RStudio', 
        'Jupyter/IPython': 'Jupyter/IPython', 
        'PyCharm': 'PyCharm', 
        'Atom': 'Vim/Emacs/Atom',
        'Visual Studio': 'Visual Studio',
        'Notepad++': 'Notepad++/Sublime', 
        'Sublime Text': 'Notepad++/Sublime', 
        'IntelliJ': 'PyCharm', 
        'Spyder': 'Spyder',
        'Visual Studio Code': 'Visual Studio',
        'Vim': 'Vim/Emacs/Atom', 
        'Other': 'Other', 
        'nteract': 'Other', 
        np.nan: 'Other',
        'Jupyter (JupyterLab, Jupyter Notebooks, etc) ': 'Jupyter/IPython', 
        ' RStudio ': 'RStudio',
        ' PyCharm ': 'PyCharm', 
        ' MATLAB ': 'MATLAB', 
        '  Spyder  ': 'Spyder', 
        '  Notepad++  ': 'Notepad++/Sublime',
        '  Sublime Text  ': 'Notepad++/Sublime', 
        ' Atom ': 'Vim/Emacs/Atom',
        ' Visual Studio / Visual Studio Code ': 'Visual Studio',
        '  Vim / Emacs  ': 'Vim/Emacs/Atom',
        'Visual Studio Code (VSCode)': 'Visual Studio'
    }
    CLOUD={
        'I have not used any cloud providers': 'None', 
        'Microsoft Azure': 'Azure',
       'Google Cloud Platform (GCP)': 'GCP', 
        'Amazon Web Services (AWS)': 'AWS',
       'IBM Cloud': 'IBM/Red Hat', 
        'Other': 'Other', 
        'Alibaba Cloud': 'Alibaba', 
        np.nan: 'None',
       ' Amazon Web Services (AWS) ': 'AWS', 
        ' Google Cloud Platform (GCP) ': 'GCP',
       ' Microsoft Azure ': 'Azure', 
        'None': 'None', 
        ' Salesforce Cloud ': 'Other',
       ' Red Hat Cloud ': 'IBM/Red Hat', 
        ' VMware Cloud ': 'Other', 
        ' Alibaba Cloud ': 'Alibaba',
       ' SAP Cloud ': 'Other', 
        ' IBM Cloud ': 'IBM/Red Hat', 
        ' Oracle Cloud ': 'Other',
       ' IBM Cloud / Red Hat ': 'IBM/Red Hat',
        ' Tencent Cloud ': 'Other'
    }
    ML_STATUS={ 
        'No (we do not use ML methods)': 'Do not use ML / Do not know',
        'I do not know': 'Do not use ML / Do not know',
        'We recently started using ML methods (i.e., models in production for less than 2 years)': 'Recently started using ML',
        'We have well established ML methods (i.e., models in production for more than 2 years)':  'Well established ML',
        'We are exploring ML methods (and may one day put a model into production)': 'Exploring ML',
        'We use ML methods for generating insights (but do not put working models into production)': 'Use ML for generating insights',
        np.nan: 'Do not use ML / Do not know',
    }
    ML_FRAMEWORKS={
        'None': 'None', 
        'Prophet': 'Prophet', 
        'Scikit-Learn': 'Scikit-learn', 
        'Keras': 'Keras', 
        'TensorFlow': 'TensorFlow',
        'Spark MLlib': 'Other', 
        'Xgboost': 'Xgboost', 
        'randomForest': 'Other', 
        'lightgbm': 'LightGBM',
        'Caret': 'Caret',
        'mlr': 'Other', 
        'PyTorch': 'PyTorch', 
        'Mxnet': 'Other', 
        'CNTK': 'Other', 
        'Caffe': 'Other', 
        'H20': 'H2O', 
        'catboost': 'CatBoost',
        'Fastai': 'Fast.ai', 
        'Other': 'Other', 
        np.nan: 'None', 
        '  Scikit-learn ': 'Scikit-learn', 
        ' RandomForest': 'Other',
        ' Xgboost ': 'Xgboost', 
        ' LightGBM ': 'LightGBM',
        '  TensorFlow ': 'TensorFlow',
        ' Keras ': 'Keras', 
        ' Caret ': 'Caret',
        ' PyTorch ': 'PyTorch', 
        ' Spark MLib ': 'Spark MLlib',
        ' Fast.ai ': 'Fast.ai', 
        ' Tidymodels ': 'Other',
        ' CatBoost ': 'CatBoost', 
        ' JAX ': 'Other', 
        ' Prophet ': 'Prophet', 
        ' H2O 3 ': 'H2O', 
        ' MXNet ': 'Other'   
    }
    
    
class Category(Enum):
    COMPENSATION=[
        'Not Disclosed', '0-10k', '10-20k', '20-30k', '30-40k', '40-50k', '50-60k', 
        '60-70k', '70-80k', '80-90k', '90-100k', '100-125k', '125-150k', '150-200k', 
        '200-250k', '300-500k'
    ]
    JOB_TITLE=[
        'Other', 'Manager/C-level', 'Product/Project Manager', 'Business Analyst', 'Data Analyst', 
        'Research Scientist/Statistician', 'Data Scientist', 'Machine Learning Engineer', 
        'Data Engineer/DBA', 'Software Engineer'
    ]  
    GENDER = ['Women', 'Men'] 
    AGE=['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-69', '70+']
    YEARS_WRITING_CODE=['None', '< 1 year', '1-3 years', '3-5 years', '5-10 years', '10+ years']
    YEARS_USING_ML=['None', '< 1 year', '1-3 years', '3-5 years', '5+ years']
    SURVEY_YEAR=[2018, 2019, 2020]
    EDUCATION=['High school', 'Some college', 'Professional', 'Bachelor’s', 'Master’s', 'Doctoral']
    PROFILES=['Beginners', 'Others', 'Modern DS', 'Coders', 'ML Veterans']

COLORS = {
    'India': '#FE9933', 
    'Brazil': '#179B3A',
    'United States': '#002366', 
    'China': '#ED2124', 
    'Average': 'blueviolet',
    'Canada': '#F60B00',
    'Data Scientist': '#13A4B4',
    'Product/Project Manager': '#D70947',
    'Software Engineer': '#E8743B', 
    'Data Analyst': '#BF399E',
    'Data Engineer/DBA': '#144B7F',
    '< 1 year': 'lightgreen', 
    '10+ years': 'green', 
    'Women': 'hotpink', 
    'Men': 'midnightblue',
    'Python': '#FEC331',
    'SQL': '#66B900',
    'R': '#2063b7',
    'C/C++': 'slateblue',
    'Basic statistical software': '#0D7036', 
    'Local or hosted development environments': '#36B5E2',
    'Visual Studio': '#349FED',
    'Jupyter/IPython': '#EC7426',
    'AWS': '#F79500',
    'GCP': '#1AA746',
    'Azure': '#3278B1',
    'Well established ML': 'dodgerblue', 
    'PyTorch': 'orangered', 
    'Scikit-learn': 'goldenrod',
    'None': 'darkblue',
    'Do not use ML / Do not know': 'slategrey',
    'Exploring ML': 'lightseagreen', 
    'Recently started using ML': 'forestgreen'
}

In [ ]:
class BaseKaggle(ABC):
    """
    Base class to handle cleaning and transformation of datasets from different years.
    """
    def __init__(self) -> None:
        self.df = None
        self.non_professionals = ['Student', 'Currently not employed', np.nan]
        self.mapping = {}
        self.questions_to_combine = []
        self.survey_year = None
   
    @property
    def questions_to_keep(self) -> List[str]:
        """
        Select which questions we should keep in the dataframe using the mapping keys
        """
        return [key for key, value in self.mapping.items()]

    def remove_non_professionals(self) -> pd.DataFrame:
        """
        Non-professionals were defined as students, unemployed and NaNs. 
        Also removed those who didn't disclose compensation.
        """
        self.df = self.df.drop(self.df[self.df['Job Title'].isin(self.non_professionals)].index)
        self.df.dropna(subset=['Yearly Compensation'], inplace=True)
        return self.df
    
    @abstractmethod
    def filter_question_columns(columns: List[str], question: str) -> List[str]:
        pass
    
    @staticmethod
    def remove_nans_from_list(answers: List[str]) -> List[str]:
        """
        This function removes all nans from a list
        """
        return [x for x in answers if pd.notnull(x)]

    def combine_answers_into_list(self, question: str) -> pd.DataFrame:
        """
        This function will create a new column in the dataframe adding 
        all answers to a list and removing nans.
        """
        filtered_columns = self.filter_question_columns(list(self.df.columns), question)
        self.df[question] = self.df[filtered_columns].values.tolist()
        self.df[question] = self.df[question].apply(self.remove_nans_from_list)
        return self.df

    def batch_combine_answers_into_list(self, questions_to_combine: List[str]) -> pd.DataFrame:
        """
        Applyes combine_answers_into_list to multiple columns
        """
        for question in questions_to_combine:
            self.combine_answers_into_list(question=question)
        return self.df
    
    def rename_columns(self) -> pd.DataFrame:
        """
        Renames columns using mapping
        """
        self.df = self.df.rename(columns=self.mapping)
        return self.df
    
    def do_mapping(self, column: str, mapping: Mapping) -> pd.DataFrame:
        """
        Maps values to have same classes accross all years
        """
        self.df[column] = self.df[column].map(mapping.value)
        return self.df
    
    def do_list_mapping(self, column: str, mapping: Mapping) -> pd.DataFrame:
        """
        Maps values to have same classes accross all years for columns that are list type
        """
        mapping_dict = mapping.value
        self.df[column] = self.df[column].apply(lambda x: [mapping_dict[val] for val in x])
        return self.df

    def add_numeric_average_compensation(self) -> pd.DataFrame:
        """
        Create a numeric value for compensation, taking the average between the max and min values for each class
        
        We are summing up the lowest and highest value for each category, and then dividing by 2.
        Some regex needed to clean the text
        """
        compensation = self.df['Yearly Compensation'].str.replace(r'(?:(?!\d|\-).)*', '').str.replace('500', '500-500').str.split('-')
        self.df['Yearly Compensation Numeric'] = compensation.apply(lambda x: (int(x[0]) + int(x[1]))/ 2) # it is calculated in thousand of dollars
        return self.df

    def add_survey_year_column(self) -> pd.DataFrame:
        """
        Adds the year the survey was taken as a column
        """
        self.df['Survey Year'] = self.survey_year
        return self.df
    
    def add_dummy_column(self) -> pd.DataFrame:
        """
        Adds Dummy = 1 to make easier to perform group by
        """
        self.df['Dummy'] = 1
        return self.df
    
    def select_questions(self) -> pd.DataFrame:
        """
        Selects only the relevant questions from each survey year
        """
        self.df = self.df[self.questions_to_keep]
        return self.df
    
    def fill_na(self, column: str, value: str) -> pd.DataFrame:
        """
        Fill column NaNs with a given value
        """
        self.df[column] = self.df[column].fillna(value)
        return self.df
   
    def calculate_profile(self, values: tuple) -> str:
        """
        This function creates profiles for professionals adapted from the work developed by Teresa Kubacka on last years survey
        https://www.kaggle.com/tkubacka/a-story-told-through-a-heatmap
        """
        years_code, years_ml = values
        if years_code in ['0-1 years', '1-2 years'] and years_ml in ['0-1 years', '1-2 years']:
            return 'Beginners'
        elif years_code in ['2-3 years', '3-10 years'] and years_ml in ['1-2 years', '2-3 years', '3-10 years']:
            return 'Modern DS'
        elif years_code == '10+ years' and years_ml in ['0-1 years', '1-2 years']:
            return 'Coders'
        elif years_code == '10+ years' and years_ml == '10+ years':
            return 'ML Veterans'
        else:
            return 'Others'

    def create_profiles(self) -> None:
        """
        This function creates a new columns with profiles for professionals adapted from the work developed by Teresa Kubacka on last years survey
        https://www.kaggle.com/tkubacka/a-story-told-through-a-heatmap
        """
        self.df['Years Writing Code Profile'] = self.df['Tenure: Years Writing Code'].map(Mapping.YEARS_WRITING_CODE_PROFILES.value)
        self.df['Years Using ML Profile'] = self.df['Tenure: Years Using Machine Learning Methods'].map(Mapping.YEARS_USING_ML_PROFILES.value)
        
        self.df['Profile'] = self.df[
            ['Years Writing Code Profile', 
             'Years Using ML Profile']
        ].apply(self.calculate_profile, axis=1)

        
    def transform(self) -> pd.DataFrame:
        """
        Process and clean the dataset
        """

        self.df.drop(0, axis=0, inplace=True)  # dropping first row (questions) from processed data

        self.batch_combine_answers_into_list(questions_to_combine=self.questions_to_combine)
        self.select_questions()
        self.rename_columns()

        self.create_profiles()
        self.do_mapping(column='Yearly Compensation', mapping=Mapping.COMPENSATION)
        self.do_mapping(column='Job Title', mapping=Mapping.JOB_TITLE)
        self.do_mapping(column='Gender', mapping=Mapping.GENDER)
        self.do_mapping(column='Age', mapping=Mapping.AGE)
        self.do_mapping(column='Education', mapping=Mapping.EDUCATION)
        self.do_mapping(column='Tenure: Years Writing Code', mapping=Mapping.YEARS_WRITING_CODE)
        self.do_mapping(column='Recommended Programming Language', mapping=Mapping.RECOMMENDED_LANGUAGE)
        self.do_mapping(column='Tenure: Years Using Machine Learning Methods', mapping=Mapping.YEARS_USING_ML)
        self.do_mapping(column='Primary Tool to Analyze Data', mapping=Mapping.PRIMARY_TOOL)
        self.do_mapping(column='Country', mapping=Mapping.COUNTRY)
        self.do_mapping(column='Machine Learning Status in Company', mapping=Mapping.ML_STATUS)
        self.do_list_mapping(column='Machine Learning Frameworks', mapping=Mapping.ML_FRAMEWORKS)
        
        self.do_list_mapping(column='Programming Languages', mapping=Mapping.LANGUAGES)
        self.do_list_mapping(column='IDEs', mapping=Mapping.IDE)
        self.do_list_mapping(column='Cloud Computing Platforms', mapping=Mapping.CLOUD)
        self.fill_na(column='Country', value='Other')

        self.remove_non_professionals()       
        self.add_numeric_average_compensation()
        self.add_survey_year_column()
        self.add_dummy_column()
                
        self.df.reset_index(drop=True, inplace=True)
    
        return self.df

In [ ]:
class Kaggle2019(BaseKaggle):
    """
    Processing and cleaning 2019 Dataset
    """
    
    
    def __init__(self) -> None:
        super().__init__()
        self.survey_year = 2019
        self.df = pd.read_csv('/kaggle/input/kaggle-survey-2019/multiple_choice_responses.csv', low_memory=False)
        self.mapping = {
            'Q1':'Age',
            'Q2':'Gender',
            'Q3':'Country',
            'Q4':'Education',
            'Q5':'Job Title',
            'Q15':'Tenure: Years Writing Code',
            'Q18':'Programming Languages',
            'Q19':'Recommended Programming Language',
            'Q16':'IDEs',
            'Q17':'Hosted Notebooks',
            'Q20':'Data Visualization Libraries',
            'Q23':'Tenure: Years Using Machine Learning Methods',
            'Q28':'Machine Learning Frameworks',
            'Q8':'Machine Learning Status in Company',
            'Q9':'Daily activities',
            'Q10':'Yearly Compensation',
            'Q29':'Cloud Computing Platforms',
            'Q30':'Cloud Computing Products',
            'Q32':'Machine Learning Products',
            'Q31':'Big Data Products',
            'Q13':'Data Science Courses',
            'Q14':'Primary Tool to Analyze Data',
            'Q12':'Media Sources',
        }
        self.questions_to_combine = [
            'Q18', 'Q16', 'Q17', 'Q20', 'Q28', 'Q9', 'Q29', 'Q30', 'Q32', 'Q31', 'Q13', 'Q12'
        ]
           
    @staticmethod
    def filter_question_columns(columns: List[str], question: str) -> List[str]:
        """
        Filters only questions that starts with the question_number and do not end with the string _OTHER_TEXT
        """
        return [col for col in columns if col.startswith(f'{question}_P') and not col.endswith('_OTHER_TEXT')]

Gráfics

In [ ]:
MetricData = namedtuple('MetricData', ['subplot_name','trace_name','y_values', 'x_values', 
                                       'subplot_y_position', 'subplot_x_position', 'highlighted_traces']) 


class BaseMetric(ABC):
    """
    Creates a plotly plot for slopegraphs
    """
    
    def __init__(
        self, 
        survey: KaggleCombinedSurvey, 
        traces_col: str, 
        y_col: str, 
        x_col: str,
        explode: bool = False
    ) -> None:
        """
        traces: the column name we want to creaate traces from
        y: the column name we will be ploting
        x: Will always be survey year for our slopegraphs.
        """
        self.traces_col = traces_col
        self.y_col = y_col
        self.x_col = x_col
        self.survey = survey
        self.traces = []
        self.explode = explode
        self.metric_df = None

    @property
    def traces_names(self) -> List[str]:
        """
        Calculate unique values of traces_col
        """
        return self.metric_df[self.traces_col].cat.categories
    
    @property
    def subplots_names(self) -> List[str]:
        """
        Calculate unique values of traces_col
        """
        return self.metric_df[self.y_col].cat.categories

    @property
    def subplots_qty(self):
        return len(self.subplots_names)
    
    @property
    def traces_qty(self):
        return len(self.traces_names)
    
    def apply_filter(self, df: pd.DataFrame, column: str, value: str) -> pd.DataFrame:
        """
        filters data for a single trace
        """
        return df[df[column] == value] 

    @abstractmethod
    def calculate(self) -> pd.DataFrame:
        """
        Group the data by y_col, perform count and convert it to a list
        Transforms absolute values into percentages
        Yeld the metrics for a given trace
        """
        pass
   
    def groupby(self, df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
        """"
        Calculates quantity per x, y and traces col
        """
        return df.groupby(columns, as_index=False)['Dummy'].sum()
    
    def join_dataframes(self, df1: pd.DataFrame, df2: pd.DataFrame, on_columns: List[str]) -> pd.DataFrame:
        return df1.set_index(on_columns).join(df2.set_index(on_columns), rsuffix='_total').reset_index()
    
    def to_categorical(self, column: str, categories: Category) -> pd.DataFrame:
        cat_dtype = pd.api.types.CategoricalDtype(categories=categories.value, ordered=True)
        self.metric_df[column] = self.metric_df[column].astype(cat_dtype)
        return self.metric_df
    
    def get_df(self):
        """
        Returns a dataframe with or without lists exploded 
        """
        if self.explode:
            return self.survey.df.explode(self.traces_col)
        else:
            return self.survey.df
        
    def get_subplots(self, highlighted_traces: List[str]) -> List[MetricData]:
        self.apply_categories()
        self.metric_df['subplot_y_position'] = self.metric_df[self.y_col].cat.codes + 1
        self.metric_df['subplot_x_position'] = 1       

        for index, row in self.metric_df.iterrows():
            filtered_df = self.apply_filter(df=self.metric_df, column=self.y_col, value=row[self.y_col])
            filtered_df = self.apply_filter(df=filtered_df, column=self.traces_col, value=row[self.traces_col])
            
            metric_data = MetricData(
                subplot_name=row[self.y_col],
                trace_name=row[self.traces_col],
                y_values=filtered_df['Metric'].values,
                x_values=filtered_df[self.x_col].values,
                subplot_y_position=row['subplot_y_position'],
                subplot_x_position=row['subplot_x_position'],
                highlighted_traces=row[self.traces_col] in highlighted_traces
            )
            self.traces.append(metric_data)

In [ ]:
class PercentageMetric(BaseMetric):
    """
    Creates a plotly plot for slopegraphs
    """   
    
    def calculate_average(self, df=pd.DataFrame) -> pd.DataFrame:
        detail = self.groupby(df=df, columns=[self.x_col, self.y_col])
        total = self.groupby(df=df, columns=[self.x_col])
        joined = self.join_dataframes(df1=detail, df2=total, on_columns=[self.x_col]) 
        joined['Metric'] = joined['Dummy'] / joined['Dummy_total'] * 100  # get percentage
        joined[self.traces_col] = 'Average'
        return joined
            
    
    def calculate(self, add_avg: bool = False) -> pd.DataFrame:
        """
        Group the data by y_col, perform count and convert it to a list
        Transforms absolute values into percentages
        Yeld the metrics for a given trace
        """
        df = self.get_df()
        detail = self.groupby(df=df, columns=[self.x_col, self.y_col, self.traces_col])
        total = self.groupby(df=df, columns=[self.x_col, self.traces_col]) 
        joined = self.join_dataframes(df1=detail, df2=total, on_columns=[self.x_col, self.traces_col]) 
        joined['Metric'] = joined['Dummy'] / joined['Dummy_total'] * 100  # get percentage
        
        if add_avg:
            avg_df = self.calculate_average(df=joined)
            joined = joined.append(avg_df)
                
        self.metric_df = joined
        return joined

    @abstractmethod
    def apply_categories(self):
        pass 

In [ ]:
class GenderProportionMetric(PercentageMetric):
    def __init__(self, survey: KaggleCombinedSurvey) -> None:
        super().__init__(survey=survey, traces_col='Country', y_col='Gender', x_col='Survey Year')
    
    def apply_categories(self):
        self.to_categorical(column='Gender', categories=Category.GENDER)
        self.to_categorical(column='Survey Year', categories=Category.SURVEY_YEAR)

In [ ]:
class BasePlot(ABC):
    """
    Creates a plotly plot for slopegraphs
    """
    
    def __init__(
        self, 
        metric: Type[BaseMetric], 
        title: str, 
        yaxes_title: str, 
        shared_yaxes: bool, 
        yticks: List[float], 
        yticks_template: str, 
        annotation_template: str,
        x_nticks: int,
        hover_template: str
    ) -> None:
    
        pyo.init_notebook_mode()
        self.metric = metric
        self.yaxes_title = yaxes_title
        self.shared_yaxes = shared_yaxes
        self.hover_template = hover_template
        self.title = title
        self.yticks = yticks
        self.yticks_template = yticks_template
        self.annotation_template = annotation_template
        self.x_nticks = x_nticks
        self.figure = go.Figure()
        self.range = (0, 0)

    def make_subplots(self) -> None:
        """
        Creates subplots in the figure and add titles
        """
        self.figure = make_subplots(
            cols=self.metric.subplots_qty, # our subplots will have the number of unique values for the select column
            rows=1, # and 1 row
            subplot_titles=self.metric.subplots_names, # Add titles to subplots
            specs=[[{'type': 'scatter'}]*self.metric.subplots_qty]*1, # Define chart type for each subplot
            shared_yaxes=self.shared_yaxes,
            shared_xaxes=True
        )

        for idx, subplot_title in enumerate(self.figure['layout']['annotations']):
            subplot_title['font'] = dict(size=14,color='grey')  # Size and color of subplot title
            subplot_title['align'] = 'left'
            subplot_title['xanchor'] = 'left'
            subplot_title['x'] = 0
            subplot_title['xref'] = 'x' if idx == 0 else f'x{idx + 1}'
            

    def update_common_layout(self) -> None:
        """
        Updates general layout characteristics
        """
        self.figure.update_layout(
            showlegend = False,
            plot_bgcolor='white',
            title_text = self.title,
            title_font_color = 'grey',
            title_font_size = 15,
            title_x=0,
            title_y=0.98,
            margin_t=130,
            margin_l=0,
            margin_r=0,
            height=600,
            width=800,
            yaxis_range=self.range
        )

    def get_yticks_text(self) -> List[str]:
        """
        Calculates the y_ticks text for charts
        """
        return [self.yticks_template.format(i) for i in self.yticks]
    
    def update_subplots_layout(self) -> None:
        """
        Updates scatter subplots layout characteristics
        """
        for subplot_idx in range(self.metric.subplots_qty):
            self.figure.update_xaxes(
                type='category',
                color='lightgrey', # to not draw to much attention to axis
                showgrid=False, 
                visible=subplot_idx == 0,  # Visible only to the first subplot
                row=1,
                nticks=self.x_nticks,               
                col=subplot_idx + 1 # Subplots start at 1 
            )
            self.figure.update_yaxes(
                showgrid=False,
                visible=subplot_idx == 0 or not self.shared_yaxes,
                title=self.yaxes_title if subplot_idx == 0 else None,  # Visible only to the first subplot
                color='grey',
                row=1, 
                col=subplot_idx + 1,
                tickvals=self.yticks, # show ticks ate 25, 50 and 75
                ticktext=self.get_yticks_text(),
                tickmode='array',
                tickfont_color='lightgrey',
                autorange=True
            )

    def line_color(self, trace: MetricData) -> str:
        """
        Sets color to the highlight color or to a tone of grey if not highlighted
        """
        return self.highlight_color(trace=trace) if trace.highlighted_traces else 'lightslategrey'
    
    def highlight_color(self, trace: MetricData) -> str:
        """
        Returns the highlight color
        """
        return COLORS[trace.trace_name]
    
    def line_width(self, trace: MetricData) -> str:
        """
        Returns the line width of traces depending if trace is highlighted or not
        """
        return 1.6 if trace.highlighted_traces else 0.6
   
    def opacity(self, trace: MetricData) -> str:
        """
        Returns the opacity depending if trace is highlighted or not
        """
        return 0.8 if trace.highlighted_traces else 0.25

    def add_trace(self, trace: MetricData) -> None:
        """
        Adds a new trace to a figure
        """
        self.figure.add_trace(
            go.Scatter(
                x=trace.x_values, 
                y=trace.y_values, 
                mode='lines',
                name=trace.trace_name,
                hoverinfo='name+text+y',
                hovertemplate=self.hover_template,
                text=trace.x_values,
                line_color=self.line_color(trace=trace),
                showlegend=False,
                opacity= self.opacity(trace=trace),
                line_shape='linear',
                line_width=self.line_width(trace=trace),
                connectgaps=True
            ), 
            trace.subplot_x_position, 
            trace.subplot_y_position
        )
    
    def get_annotation_text(self, trace: MetricData, idx: int) -> str:
        """
        Calculates the annotation text to be added to the plot
        """
        if trace.subplot_y_position == 1 and idx == 0:
            template = '{}<br>' + f'{self.annotation_template}'
            return template.format(trace.trace_name, trace.y_values[idx])
        else:
            return self.annotation_template.format(trace.y_values[idx])
        
    def add_annotations(self) -> None:
        """
        Adds annotations to the plot
        """
        for trace in self.metric.traces:
            if trace.highlighted_traces:
                # Add left annotation
                self.figure.add_annotation(
                    xref=f'x{trace.subplot_y_position}',
                    yref=f'y{trace.subplot_y_position}',
                    font=dict(
                        size=11,
                        color=self.highlight_color(trace=trace)
                    ),
                    opacity=0.8,
                    align='center',
                    yshift=0,
                    xshift=-3,
                    xanchor='right',
                    x=trace.x_values.codes[0], 
                    y=trace.y_values[0],
                    text=self.get_annotation_text(trace=trace, idx=0),
                    showarrow=False
                    )
                # Add right annotation
                self.figure.add_annotation(
                    xref=f'x{trace.subplot_y_position}',
                    yref=f'y{trace.subplot_y_position}',
                    font=dict(
                        size=11,
                        color=self.highlight_color(trace=trace)
                    ),
                    opacity=0.8,
                    align='center',
                    yshift=0,
                    xshift=3,
                    xanchor='left',
                    x=trace.x_values.codes[-1], 
                    y=trace.y_values[-1],
                    text=self.get_annotation_text(trace=trace, idx=-1),
                    showarrow=False
                    )

    def add_subplot_axis_annotation(self) -> None:
        """
        Add subplot axis annotation
        """
        self.figure.add_annotation(
            xref="x", 
            yref="paper",
            font=dict(
                size=14,
                color='lightgrey'
            ),
            align='left',
            x=0, 
            xanchor='left',
            y=1.05,
            yanchor='bottom',
            text=f'{self.metric.y_col}',
            showarrow=False
        )
        
    def add_source_annotation(self) -> None:
        """
        Add source annotation
        """
        self.figure.add_annotation(
            xref="paper", 
            yref="paper",
            font=dict(
                size=11,
                color='lightgrey'
            ),
            align='left',
            x=-0.07, 
            xanchor='left',
            y=-0.13,
            yanchor='bottom',
            text='<b>Source:</b> Kaggle surveys from 2018 to 2020.',
            showarrow=False
        )
        
    def add_data(self) -> None:
        """
        Adds a trace to the figure following the same standard for each trace
        """
        # Add all non-highlighted traces.
        for trace in self.metric.traces:
            self.add_trace(trace=trace)
            self.update_range(data=trace.y_values)

        
    def update_range(self, data: List[float]) -> None:
        """
        Updates the range to be 90% of minimum values and 110% of maximum value of all traces
        """
        if len(data) == 0:
            return self.range
        
        max_range = max(data) * 1.2
        min_range = min(data) * 0.8
        self.range = (self.range[0], max_range) if max_range > self.range[1] else self.range 
        self.range = (min_range, self.range[1]) if min_range < self.range[0] else self.range
        
    def show(self) -> None:
        """
        Renders and shows the plot
        """
        self.make_subplots()
        self.update_common_layout()
        self.add_data()
        self.add_annotations()
        self.add_subplot_axis_annotation()
        self.update_subplots_layout()
        self.add_source_annotation()
        self.figure.show()

#### Preprocessamento

In [173]:
dfpt1['nivel'].drop_duplicates()

0         NaN
482    Sênior
483     Pleno
484    Júnior
Name: nivel, dtype: object

In [ ]:
class columns_adjustments(Enum):
    RENAMES = {
        "('P0', 'id')":"id",
        "('P1_a ', 'Idade')":"idade",
        "('P1_a_a ', 'Faixa idade')":"faixa_idade",
        "('P1_b ', 'Genero')": "genero",
        "('P1_e_b ', 'Regiao onde mora')":"regiao_moradia",
        "('P1_g_b ', 'Regiao de origem')":"regiao_origem",
        "('P1_g_c ', 'Mudou de Estado?')":"mudou",
        "('P1_h ', 'Nivel de Ensino')":"ensino",
        "('P1_i ', 'Área de Formação')":"formacao",
        "('P2_b ', 'Setor')":"setor",
        "('P2_g ', 'Nivel')":"nivel",
        "('P2_h ', 'Faixa salarial')":"faixa_salarial",
        "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')":"tempo_dados",
        "('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')":"tempo_dev",
        "('P2_k ', 'Você está satisfeito na sua empresa atual?')":"satisfacao",
        "('P2_q ', 'Atualmente qual a sua forma de trabalho?')":"formato_trabalho",
        "('P6_h ', 'Em qual das opções abaixo você gasta a maior parte do seu tempo?')":"de_tempo",
        "('P7_d ', 'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?')":"da_tempo",
        "('P8_d ', 'Em qual das opções abaixo você gasta a maior parte do seu tempo no trabalho?')":"ds_tempo",
    }

    SELECT = [
        "id",
        "idade",
        "faixa_idade",
        "genero",
        "regiao_moradia",
        "regiao_origem",
        "mudou",
        "ensino",
        "formacao",
        "setor",
        "nivel",
        "faixa_salarial",
        "tempo_dados",
        "tempo_dev",
        "satisfacao",
        "formato_trabalho",
        "de_tempo",
        "da_tempo",
        "ds_tempo",
    ]

    FAIXA_SALARIAL = {
        "Menos de R$ 1.000/mês":"0-2k",
        "de R$ 1.001/mês a R$ 2.000/mês":"0-2k",
        "de R$ 2.001/mês a R$ 3000/mês":"2-4k",
        "de R$ 3.001/mês a R$ 4.000/mês":"2-4k",
        "de R$ 4.001/mês a R$ 6.000/mês":"4-6k",
        "de R$ 6.001/mês a R$ 8.000/mês":"6-8k",
        "de R$ 8.001/mês a R$ 12.000/mês":"8-12k",
        "de R$ 12.001/mês a R$ 16.000/mês":"12-16k",
        "de R$ 16.001/mês a R$ 20.000/mês":"16-25k",
        "de R$ 20.001/mês a R$ 25.000/mês":"16-25k",
        "de R$ 25.001/mês a R$ 30.000/mês":"25k +",
        "de R$ 30.001/mês a R$ 40.000/mês":"25k +",
        "Acima de R$ 40.001/mês":"25k +",
        np.nan:"None"
    }

    SALARIO_NUM = {
        "Menos de R$ 1.000/mês":1,
        "de R$ 1.001/mês a R$ 2.000/mês":1.5,
        "de R$ 2.001/mês a R$ 3000/mês":2.5,
        "de R$ 3.001/mês a R$ 4.000/mês":3.5,
        "de R$ 4.001/mês a R$ 6.000/mês":5,
        "de R$ 6.001/mês a R$ 8.000/mês":7,
        "de R$ 8.001/mês a R$ 12.000/mês":10,
        "de R$ 12.001/mês a R$ 16.000/mês":14,
        "de R$ 16.001/mês a R$ 20.000/mês":18,
        "de R$ 20.001/mês a R$ 25.000/mês":22.5,
        "de R$ 25.001/mês a R$ 30.000/mês":27.5,
        "de R$ 30.001/mês a R$ 40.000/mês":35,
        "Acima de R$ 40.001/mês":40,
        np.nan:"None"
    }
    
    NIVEL = {
        np.nan:"None",
        "Júnior":"Jr",
        "Pleno":"Pl",
        "Sênior":"Sr"
    }

    ENSINO = {
        "Não tenho graduação formal":"Autodidata",
        "Estudante de Graduação":"Estudante",
        "Graduação/Bacharelado":"Graduação",
        "Pós-graduação":"Especialização",
        "Mestrado":"Mestrado",
        "Doutorado ou Phd":"Doutorado",
        "Prefiro não informar":"None",
        np.nan:"None"
    }

    TEMPO_DEV = {
        "Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados": "None",
        "Menos de 1 ano":"0-1 ano",
        "de 1 a 2 anos":"1-2 anos",
        "de 2 a 3 anos":"2-3 anos",
        "de 4 a 5 anos":"3-5 anos",
        "de 6 a 10 anos":"5-10 anos",
        "Mais de 10 anos":"10+ anos",
        np.nan:"None"
    }

    TEMPO_DADOS = {
        "Não tenho experiência na área de dados": "None",
        "Menos de 1 ano":"0-1 ano",
        "de 1 a 2 anos":"1-2 anos",
        "de 2 a 3 anos":"2-3 anos",
        "de 4 a 5 anos":"3-5 anos",
        "de 6 a 10 anos":"5-10 anos",
        "Mais de 10 anos":"10+ anos",
        np.nan:"None"

    }

    SATISFACAO = {
        1:"Contente",
        0:"Descontente",
        np.nan:"None"
    }

    MUDOU = {
        1:"Sim",
        0:"Não",
        np.nan:"None"
    }

    FORMATO_TRABALHO = {
        "Modelo 100% presencial":"Presencial",
        "Modelo híbrido flexível (o funcionário tem liberdade para escolher quando estar no escritório presencialmente)":"Flexível",
        "Modelo híbrido com dias fixos de trabalho presencial":"Híbrido",
        "Modelo 100% remoto":"Remoto",
        np.nan:"None"
    }

    FORMACAO = {
        "Computação / Engenharia de Software / Sistemas de Informação/ TI": "Tecnologia",
        "Economia/ Administração / Contabilidade / Finanças":"Negócios",
        "Química / Física":"Naturais e Matemáticas",
        "Estatística/ Matemática / Matemática Computacional":"Naturais e Matemáticas",
        "Outras Engenharias":"Engenharias",
        "Ciências Sociais":"Comunicação e Sociais",
        "Marketing / Publicidade / Comunicação / Jornalismo":"Comunicação e Sociais",
        "Ciências Biológicas/Farmácia/Medicina/Área da Saúde": "Biologia e Saúde",
        "Outras":"Outras"
    }

    SETOR = {
        "Varejo":"Varejo",
        "Tecnologia/Fábrica de Software":"Tecnologia",
        "Internet/Ecommerce":"Tecnologia",
        "Finanças ou Bancos": "Finanças ou Bancos",
        "Seguros ou Previdência": "Finanças ou Bancos",
        "consultoria":"Serviços",
        "Marketing":"Serviços",
        "Serviços":"Serviços",
        "Entretenimento ou Esportes":"Serviços",
        "Telecomunicação":"Infraestrutura",
        "Setor Alimentício":"Infraestrutura",
        "Energia":"Infraestrutura",
        "Logística":"Infraestrutura",
        "Imobiliário":"Infraestrutura",
        "Construção Civil":"Infraestrutura",
        "Indústria":"Indústria",
        "Agronegócios":"Indústria",
        "Setor Automotivo":"Indústria",
        "Outro":"Outros",
        "Área da Saúde":"Outros",
        "Educação":"Outros",
        "Setor Público":"Outros",
        np.nan:"None"
    }

    class categorias(Enum):
        FAIXA_SALARIAL = [
        "None", "0-2k","2-4k","4-6k","6-8k","8-12k","12-16k","16k +"
        ]
        
        ENSINO = [
        "None","Autodidata","Estudante","Graduação","Especialização","Mestrado","Doutorado"
        ]

        TEMPO_DEV = [
        "None","0-1 ano","1-2 anos","2-3 anos","3-5 anos","5-10 anos","10+ anos"
        ]

        TEMPO_DADOS = [
        "None","0-1 ano","1-2 anos","2-3 anos","3-5 anos","5-10 anos","10+ anos"
        ]

        SATISFACAO = [
        "None","Contente","Descontente"
        ]

        FORMATO_TRABALHO = [
        "None", "Presencial", "Flexível", "Híbrido", "Remoto"
        ]

        FORMACAO = [
        "Tecnologia","Negócios","Naturais e Matemáticas","Engenharias","Comunicação e Sociais","Biologia e Saúde","Outras"
        ]

        SETOR = [
        "None","Varejo","Tecnologia","Finanças ou Bancos","Serviços","Infraestrutura","Indústria","Outros"
        ]

In [ ]:
class dataset_threatment(ABC):

    def columns_selection(self, cols, mapping: Mapping) -> pd.DataFrame:
        """
        Renomeia e seleciona as colunas para análise
        """
        self.df.rename(columns = mapping, inplace = True)
        self.df = self.df[cols]
        return self.df

    def padronizar_valores(self, column: str, mapping: Mapping) -> pd.DataFrame:
        """
        Padroniza os valores contido nas colunas, para todas as colunas
        """
        self.df[column] = self.df[column].map(mapping.value)
        return self.df

    def salario_numerico(self) -> pd.DataFrame:
        """
        Create a numeric value for compensation, taking the average between the max and min values for each class
        
        We are summing up the lowest and highest value for each category, and then dividing by 2.
        Some regex needed to clean the text
        """
        compensation = self.df['faixa_salarial'].str.replace(r'(?:(?!\d|\-).)*', '').str.replace('500', '500-500').str.split('-')
        self.df['salario'] = self.padronizar_valores(
            column='faixa_salarial',
            mapping=columns_adjustments.FAIXA_IDADE
            )
        return self.df

    def transform(self) -> pd.DataFrame:
        """
        Processa os ajustes definidos
        """
        self.columns_selection(
            cols = columns_adjustments.SELECT, 
            mapping = columns_adjustments.RENAMES
            )
        
        self.salario_numerico()

        self.padronizar_valores(
            column='faixa_idade',
            mapping=columns_adjustments.FAIXA_IDADE
            )
        self.padronizar_valores(
            column='ensino', 
            mapping=columns_adjustments.JOB_TITLE
            )
        self.padronizar_valores(
            column='formacao', 
            mapping=columns_adjustments.GENDER
            )
        self.padronizar_valores(
            column='setor', 
            mapping=columns_adjustments.AGE
            )
        self.padronizar_valores(
            column='nivel', 
            mapping=columns_adjustments.EDUCATION
            )
        self.padronizar_valores(
            column='faixa_salarial', 
            mapping=columns_adjustments.YEARS_WRITING_CODE
            )
        self.padronizar_valores(
            column='tempo_dados', 
            mapping=columns_adjustments.RECOMMENDED_LANGUAGE
            )
        self.padronizar_valores(
            column='tempo_dev', 
            mapping=columns_adjustments.RECOMMENDED_LANGUAGE
            )
        self.padronizar_valores(
            column='satisfacao', 
            mapping=columns_adjustments.RECOMMENDED_LANGUAGE
            )
        self.padronizar_valores(
            column='formato_trabalho', 
            mapping=columns_adjustments.RECOMMENDED_LANGUAGE
            )
        self.padronizar_valores(
            column='de_tempo', 
            mapping=columns_adjustments.RECOMMENDED_LANGUAGE
            )
        self.padronizar_valores(
            column='da_tempo', 
            mapping=columns_adjustments.RECOMMENDED_LANGUAGE
            )
        self.padronizar_valores(
            column='ds_tempo', 
            mapping=columns_adjustments.RECOMMENDED_LANGUAGE
            )
        """
        PAREI AQUI
        """

        self.do_list_mapping(column='Programming Languages', mapping=Mapping.LANGUAGES)
        self.do_list_mapping(column='IDEs', mapping=Mapping.IDE)
        self.do_list_mapping(column='Cloud Computing Platforms', mapping=Mapping.CLOUD)
        self.fill_na(column='Country', value='Other')

        self.remove_non_professionals()       
        self.add_numeric_average_compensation()
        self.add_survey_year_column()
        self.add_dummy_column()
                
        self.df.reset_index(drop=True, inplace=True)

    def ajustes_finais(self) -> pd.DataFrame:
        """
        Ajustes finais no dataset
        """
        self.df['regiao_origem'] = np.where(self.df['regiao_origem'].isna(), self.df['regiao_moradia'], self.df['regiao_origem'])
        # definir o que é DS, DA e DE
        # Filtrar o quem não está nesses grupos
        # Mapear demais filtros finais